# Build the logbook for Hologram with empty or filters

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : September 6th 2023
- last update : September 6th 2023


Works directory per directory (day by day)

For USDF

In [1]:
import sys
sys.path

['/sdf/data/rubin/user/dagoret/AuxTelComm/notebooks_usdf/runspectractor_standalone/2023_09',
 '/home/d/dagoret/repos/repos_w_2023_35/analysis_tools/python',
 '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/cp_pipe/g857a3418e4+7df23b908a/python',
 '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/shapelet/g44018dc512+ea6441928c/python',
 '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/meas_modelfit/g93c4d6e787+79b822255a/python',
 '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/scarlet_extensions/g9d18589735+d445b4966c/lib/python',
 '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/proxmin/g33b4157f25/lib/python',
 '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/scarlet/g63ff1f842d+4083830bf8/lib/python',
 '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/meas_extensions_scarlet/g943cb464a2+f79590064d/python',
 '/opt/lsst/software/stack/stack/miniconda3-p

In [2]:
! ls /home/d/dagoret/rubin-user/AuxTelComm/notebooks_usdf/GeneratepseudoPostISRCCD/my_postisrccd_img

20230117  20230119  20230202  20230705	20230718  20230801  20230815
20230118  20230131  20230704  20230706	20230720  20230802  20230817


In [3]:
!ls /home/d/dagoret/rubin-user/DATA/AuxtelData2023/my_postisrccd_img_forspectractor_2023/empty~holo4_003


20230117  20230119  20230202  20230705	20230718  20230801  20230815
20230118  20230131  20230704  20230706	20230720  20230802  20230817


# Select the combination DATE, FILTER, DISPERSER

In [4]:
DATE = '20230817'
filterdispersernames = ["empty~holo4_003","BG40~holo4_003","FELH0600~holo4_003","SDSSg~holo4_003",\
                        "empty~ronchi170lpmm","BG40~ronchi170lpmm","FELH0600~ronchi170lpmm","SDSSg~ronchi170lpmm",\
                        "empty~holo4_001"
                       ]
filterdispersername = filterdispersernames[0]
version="v7.0"  # new orientation

# If position need to be specified add it in this dictionnary

In [5]:
# reconstructed / good /xpos /ypos

if version == "v7.0":
    info={\

          # with red filter
          'exposure_2023031600351_postisrccd.fits':(1,1,300.,1700),\
         }
        

# Imports

In [6]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re

plt.rcParams["figure.figsize"] = (16,4)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [7]:
from astropy.io import fits

In [8]:
import imp
MODULE_EXTENSIONS = ('.py', '.pyc', '.pyo')

/tmp/ipykernel_12681/3547059130.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [9]:
def package_contents(package_name):
    file, pathname, description = imp.find_module(package_name)
    if file:
        raise ImportError('Not a package: %r', package_name)
    # Use a set because some may be both source and compiled.
    return set([os.path.splitext(module)[0]
        for module in os.listdir(pathname)
        if module.endswith(MODULE_EXTENSIONS)])

In [10]:
package_contents("spectractor")

{'__init__',
 '_version',
 'astrometry',
 'config',
 'logbook',
 'parameters',
 'tools'}

In [11]:
package_contents("spectractor/extractor")

{'__init__',
 'background',
 'chromaticpsf',
 'dispersers',
 'extractor',
 'images',
 'psf',
 'spectroscopy',
 'spectrum',
 'targets'}

# Initialisation

In [12]:
dir_logbooks="logbooks"

In [13]:
import os
if not os.path.exists(dir_logbooks):
    os.makedirs(dir_logbooks)

## Get inputs files and sort them according date and number

In [14]:
home=os.getenv("HOME")

In [15]:
image_path=filterdispersername
logbook_csv=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_"+ DATE +"_" + version + ".csv")
logbook_xlsx=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_" +DATE +"_" + version + ".xlsx")

# Path

In [16]:
def GetPaths(computer_name):
    """
    """
    
    if computer_name == "HOSTCC":
        
        path_auxtel="/sps/lsst/groups/auxtel"
        path_spectractor=os.path.join(path_auxtel,"softs/github/desc/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
    
        path_images=os.path.join(path_auxtel,"data/hack_usdf/my_postisrccd_img_forspectractor_2023/"+filterdispersername+"/"+DATE)
        if configdir == "":
            path_output_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+DATE)
        else:
            path_output_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir+"/"+DATE)
            path_topoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir)
            path_toptopoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername)
            path_toptoptopoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode)
        
    elif computer_name == "LAPTOP1":
        path_auxtel="/Users/dagoret/DATA/AuxTelData2023"
        path_spectractor=os.path.join(path_auxtel,"/Users/dagoret/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        path_images=os.path.join(path_auxtel,"data/2023/"+filterdispersername+"/"+DATE)
        if configdir == "":
            path_output_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+DATE)
        else:
            path_output_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir+"/"+DATE)
            path_topoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir)
            path_toptopoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername)
            path_toptoptopoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode)
        
        
    elif computer_name == "WORKIMAC":
        path_auxtel="/Users/sylvie/DATA/AuxTelData2023"
        path_spectractor=os.path.join(path_auxtel,"/Users/sylvie/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        path_images=os.path.join(path_auxtel,"data/2023/"+filterdispersername+"/"+DATE)
        if configdir == "":
            path_output_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+DATE)
        else:
            path_output_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir+"/"+DATE)
            path_topoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir)
            path_toptopoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername)
            path_toptoptopoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode)
            
            
        
    elif computer_name == "HOMEIMAC":
        path_auxtel="/Volumes/Backup2020/DATA"
        path_spectractor=os.path.join(path_auxtel,"MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(".","config")
        path_images=os.path.join(path_auxtel,"hack_usdf/my_postisrccd_img_forspectractor_2023/"+filterdispersername+"/"+DATE)
        
        # /Volumes/Backup2020/DATA/AuxTelDATA2023/data/2023/OutputSpectractor
        
        if configdir == "":
            path_output_spectractor=os.path.join(path_auxtel,"AuxTelDATA2023/data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+DATE)
        else:
            path_output_spectractor=os.path.join(path_auxtel,"AuxTelDATA2023/data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir+"/"+DATE)
            path_topoutput_spectractor=os.path.join(path_auxtel,"AuxTelDATA2023/data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir)
            path_toptopoutput_spectractor=os.path.join(path_auxtel,"AuxTelDATA2023/data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername)
            path_toptoptopoutput_spectractor=os.path.join(path_auxtel,"AuxTelDATA2023/data/2023/OutputSpectractor/"+imagemode)
    # /home/d/dagoret/rubin-user/DATA/AuxtelData2023/my_postisrccd_img_forspectractor_2023/empty~holo4_003
    elif computer_name == "USDF":
        path_auxtel="/home/d/dagoret/rubin-user/DATA"
        path_spectractor=os.path.join("/home/d/dagoret","repos/repos_w_2023_35/Spectractor")
        path_spectractor_config=os.path.join(".","config")
        path_images=os.path.join(path_auxtel,"AuxtelData2023/my_postisrccd_img_forspectractor_2023/"+filterdispersername+"/"+DATE)
        
        # /Volumes/Backup2020/DATA/AuxTelDATA2023/data/2023/OutputSpectractor
        
        if configdir == "":
            path_output_spectractor=os.path.join(path_auxtel,"AuxTelDATA2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+DATE)
        else:
            path_output_spectractor=os.path.join(path_auxtel,"AuxTelDATA2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir+"/"+DATE)
            path_topoutput_spectractor=os.path.join(path_auxtel,"AuxTelDATA2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir)
            path_toptopoutput_spectractor=os.path.join(path_auxtel,"AuxTelDATA2023/OutputSpectractor/"+imagemode+"/"+filterdispersername)
            path_toptoptopoutput_spectractor=os.path.join(path_auxtel,"AuxTelDATA2023/OutputSpectractor/"+imagemode)
        
        
        
    else:
        print(f"Unknown computer {computer_name}")
        
              
    return path_auxtel,path_spectractor,path_spectractor_config,path_images,path_toptoptopoutput_spectractor, path_toptopoutput_spectractor, path_topoutput_spectractor, path_output_spectractor
        

In [17]:
configdir = " "
imagemode = " "
mycomputer_name = "USDF"
path_auxtel,path_spectractor,path_spectractor_config,path_images,path_toptoptopoutput_spectractor, path_toptopoutput_spectractor, path_topoutput_spectractor, path_output_spectractor  = GetPaths(mycomputer_name)

In [18]:
path_auxtel

'/home/d/dagoret/rubin-user/DATA'

In [19]:
#  /home/d/dagoret/rubin-user/DATA/AuxtelData2023/my_postisrccd_img_forspectractor_2023/empty~holo4_003
path_images

'/home/d/dagoret/rubin-user/DATA/AuxtelData2023/my_postisrccd_img_forspectractor_2023/empty~holo4_003/20230817'

# Check the images exists

In [20]:
dir_images=path_images

In [21]:
dir_images

'/home/d/dagoret/rubin-user/DATA/AuxtelData2023/my_postisrccd_img_forspectractor_2023/empty~holo4_003/20230817'

In [22]:
! ls /home/d/dagoret/rubin-user/DATA/AuxtelData2023/my_postisrccd_img_forspectractor_2023/empty~holo4_003/20230815

exposure_2023081500089_pseudo-postisrccd.fits
exposure_2023081500092_pseudo-postisrccd.fits
exposure_2023081500093_pseudo-postisrccd.fits


In [23]:
all_files=os.listdir(dir_images)

In [24]:
all_files

['exposure_2023081700389_pseudo-postisrccd.fits',
 'exposure_2023081700228_pseudo-postisrccd.fits',
 'exposure_2023081700171_pseudo-postisrccd.fits',
 'exposure_2023081700170_pseudo-postisrccd.fits',
 'exposure_2023081700357_pseudo-postisrccd.fits',
 'exposure_2023081700432_pseudo-postisrccd.fits',
 'exposure_2023081700277_pseudo-postisrccd.fits',
 'exposure_2023081700269_pseudo-postisrccd.fits',
 'exposure_2023081700319_pseudo-postisrccd.fits',
 'exposure_2023081700314_pseudo-postisrccd.fits',
 'exposure_2023081700185_pseudo-postisrccd.fits',
 'exposure_2023081700403_pseudo-postisrccd.fits',
 'exposure_2023081700251_pseudo-postisrccd.fits',
 'exposure_2023081700332_pseudo-postisrccd.fits',
 'exposure_2023081700147_pseudo-postisrccd.fits',
 'exposure_2023081700266_pseudo-postisrccd.fits',
 'exposure_2023081700278_pseudo-postisrccd.fits',
 'exposure_2023081700282_pseudo-postisrccd.fits',
 'exposure_2023081700384_pseudo-postisrccd.fits',
 'exposure_2023081700300_pseudo-postisrccd.fits',


In [25]:
all_selected_files = []
for filename in all_files:
    if re.search(".fits$",filename):
        all_selected_files.append(filename)
N=len(all_selected_files)

In [26]:
all_selected_files=np.array(all_selected_files)

### Sort files

In [27]:
def file_tag_forsorting(filename):
    #m=re.findall('^Cor_holo4_003_.*([0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]_.*)[.]fits$',filename)
    m=re.findall('exposure.*_(.*)_pseudo-postisrccd[.]fits$',filename)
    
    return m[0]

In [28]:
file_tag_forsorting('exposure_2022031700310_pseudo-postisrccd.fits')

'2022031700310'

In [29]:
#all_tags = [ file_tag_forsorting(filename)[0] for filename in all_selected_files]
all_tags = [ file_tag_forsorting(filename) for filename in all_selected_files]
all_tags=np.array(all_tags)
all_tags

array(['2023081700389', '2023081700228', '2023081700171', '2023081700170',
       '2023081700357', '2023081700432', '2023081700277', '2023081700269',
       '2023081700319', '2023081700314', '2023081700185', '2023081700403',
       '2023081700251', '2023081700332', '2023081700147', '2023081700266',
       '2023081700278', '2023081700282', '2023081700384', '2023081700300',
       '2023081700356', '2023081700343', '2023081700166', '2023081700199',
       '2023081700328', '2023081700301', '2023081700347', '2023081700194',
       '2023081700318', '2023081700399', '2023081700346', '2023081700398',
       '2023081700304', '2023081700180', '2023081700374', '2023081700370',
       '2023081700371', '2023081700428', '2023081700252', '2023081700148',
       '2023081700417', '2023081700418', '2023081700256', '2023081700388',
       '2023081700431', '2023081700283', '2023081700375', '2023081700385',
       '2023081700210', '2023081700361', '2023081700413', '2023081700144',
       '2023081700181', '

In [30]:
indexes=np.argsort(all_tags)

In [31]:
sorted_files=all_selected_files[indexes]

In [32]:
sorted_files

array(['exposure_2023081700144_pseudo-postisrccd.fits',
       'exposure_2023081700147_pseudo-postisrccd.fits',
       'exposure_2023081700148_pseudo-postisrccd.fits',
       'exposure_2023081700166_pseudo-postisrccd.fits',
       'exposure_2023081700167_pseudo-postisrccd.fits',
       'exposure_2023081700170_pseudo-postisrccd.fits',
       'exposure_2023081700171_pseudo-postisrccd.fits',
       'exposure_2023081700180_pseudo-postisrccd.fits',
       'exposure_2023081700181_pseudo-postisrccd.fits',
       'exposure_2023081700184_pseudo-postisrccd.fits',
       'exposure_2023081700185_pseudo-postisrccd.fits',
       'exposure_2023081700194_pseudo-postisrccd.fits',
       'exposure_2023081700195_pseudo-postisrccd.fits',
       'exposure_2023081700198_pseudo-postisrccd.fits',
       'exposure_2023081700199_pseudo-postisrccd.fits',
       'exposure_2023081700209_pseudo-postisrccd.fits',
       'exposure_2023081700210_pseudo-postisrccd.fits',
       'exposure_2023081700213_pseudo-postisrccd

In [33]:
N=len(sorted_files)

# Build the logbook from the headers

In [34]:
idx=0
#ccdir='/sps/lsst/groups/auxtel/data/2021/holo/FlipCleans'
ccdir=dir_images
all_dates=[]
all_files=[]
all_seqnum=[]
all_subdir=[]
all_indexes=[]
all_targets=[]
all_filters=[]
all_disperser=[]
all_airmass=[]
all_exposures=[]
all_focus=[]
all_seeing=[]
all_P=[]
all_T=[]
all_H=[]
all_W=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header 
    all_indexes.append(idx)
    all_seqnum.append(hdr0['SEQNUM'])
    all_files.append(filename)
    all_subdir.append(ccdir)
    targetname=hdr0['OBJECT']
    all_targets.append(targetname)
    all_dates.append(hdr0['DATE-OBS'])
    all_airmass.append(hdr0['AMSTART'])
    all_filters.append(hdr0['FILTER'])
    all_disperser.append(hdr0['GRATING'])
    all_exposures.append(hdr0['EXPTIME'])
    all_focus.append(hdr0['FOCUSZ'])
    all_seeing.append(hdr0['SEEING'])
    all_P.append(hdr0['PRESSURE'])
    all_T.append(hdr0['AIRTEMP'])
    all_H.append(hdr0['HUMIDITY'])
    all_W.append(hdr0['WINDSPD'])
       
    
    idx+=1
    

# Make pandas dataframe

## Create pandas dataframe

In [35]:
df=pd.DataFrame()

In [36]:
len(all_airmass)

79

In [37]:
N=len(all_indexes)

## Fill with container info

In [38]:
df['index']=all_indexes
df['date']=all_dates
df['path']=all_subdir
df['file']=all_files
df['num']=all_seqnum
df['object']=all_targets
df['filter']=all_filters
df['disperser']=all_disperser
df['airmass']=all_airmass
df['exposure']=all_exposures
df['focus']=all_focus
df['seeing']=all_seeing
df['pressure']=all_P
df['temperature']=all_T
df['humidity']=all_H
df['windspd']=all_W

## Add dictionnary info

### Initialise new columns

In [39]:
Obj_posXpix=np.full(N,np.nan)
Obj_posYpix=np.full(N,np.nan)
Dx=np.full(N,np.nan)
Dy=np.full(N,np.nan)
run=np.full(N,np.nan)
quality=np.full(N,np.nan)
skip=np.full(N,"    ")

### Fill dictionnary info

In [40]:
# loop on rows index
for idx in np.arange(N):
    filename=all_files[idx]
    # check if filename is in dictionnary
    if filename in info.keys():
        (run_flag,quality_flag,xpos,ypos)=info[filename]
        Obj_posXpix[idx]=int(xpos)
        Obj_posYpix[idx]=int(ypos)
        run[idx]=run_flag
        if quality_flag==1:
            skip[idx] = "    "
            quality[idx]=1
        else:
            skip[idx] = "skip"
            quality[idx]=0

In [41]:
df['Obj-posXpix']=Obj_posXpix
df['Obj-posYpix']=Obj_posYpix
df['Dx']=Dx
df['Dy']=Dy
df['skip']=skip
df['run']=run
df['quality']=quality

In [42]:
pd.set_option('display.max_rows', None)

In [43]:
df

,index,date,path,file,num,object,filter,disperser,airmass,exposure,...,temperature,humidity,windspd,Obj-posXpix,Obj-posYpix,Dx,Dy,skip,run,quality
0,0,2023-08-17T23:42:04.293,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023081700144_pseudo-postisrccd.fits,144,HD185975,empty_1,holo4_003,1.903331,30.0,...,4.0500,39.362499,5.2542,NaN,NaN,NaN,NaN,,NaN,NaN
1,1,2023-08-17T23:43:56.231,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023081700147_pseudo-postisrccd.fits,147,HD185975,empty_1,holo4_003,1.902132,30.0,...,4.1500,34.474998,4.4202,NaN,NaN,NaN,NaN,,NaN,NaN
2,2,2023-08-17T23:44:39.311,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023081700148_pseudo-postisrccd.fits,148,HD185975,empty_1,holo4_003,1.901747,30.0,...,4.1500,34.150002,5.0874,NaN,NaN,NaN,NaN,,NaN,NaN
3,3,2023-08-18T00:01:25.370,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023081700166_pseudo-postisrccd.fits,166,HD146233,empty_1,holo4_003,1.096033,30.0,...,4.1875,28.112499,3.9198,NaN,NaN,NaN,NaN,,NaN,NaN
4,4,2023-08-18T00:02:08.272,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023081700167_pseudo-postisrccd.fits,167,HD146233,empty_1,holo4_003,1.096667,30.0,...,4.1500,29.125000,4.7538,NaN,NaN,NaN,NaN,,NaN,NaN
5,5,2023-08-18T00:03:54.913,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023081700170_pseudo-postisrccd.fits,170,HD146233,empty_1,holo4_003,1.098255,30.0,...,4.0500,31.025000,4.4202,NaN,NaN,NaN,NaN,,NaN,NaN
6,6,2023-08-18T00:04:37.826,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023081700171_pseudo-postisrccd.fits,171,HD146233,empty_1,holo4_003,1.098928,30.0,...,4.0500,32.037498,4.5870,NaN,NaN,NaN,NaN,,NaN,NaN
7,7,2023-08-18T00:09:47.082,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023081700180_pseudo-postisrccd.fits,180,HD146233,empty_1,holo4_003,1.104090,30.0,...,4.2250,24.225000,4.4202,NaN,NaN,NaN,NaN,,NaN,NaN
8,8,2023-08-18T00:10:29.990,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023081700181_pseudo-postisrccd.fits,181,HD146233,empty_1,holo4_003,1.104852,30.0,...,4.1500,25.075001,5.0040,NaN,NaN,NaN,NaN,,NaN,NaN
9,9,2023-08-18T00:12:16.474,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023081700184_pseudo-postisrccd.fits,184,HD146233,empty_1,holo4_003,1.106748,30.0,...,3.9500,28.250000,4.0866,NaN,NaN,NaN,NaN,,NaN,NaN


# Make logbook


In [44]:
N=len(df)
N

79

In [45]:
if N>0:
#    df.to_excel(logbook_xlsx)
    df.to_csv(logbook_csv)

In [46]:
logbook_csv

'logbooks/auxtellogbook_empty~holo4_003_20230817_v7.0.csv'